# Day 3: Execution Algorithms - TWAP, VWAP & Implementation Shortfall

**Week 21: Market Microstructure**

---

## Learning Objectives

1. Understand the purpose and mechanics of execution algorithms
2. Implement Time-Weighted Average Price (TWAP) algorithm
3. Implement Volume-Weighted Average Price (VWAP) algorithm
4. Understand and implement Implementation Shortfall (IS) algorithms
5. Compare algorithm performance using various metrics
6. Learn when to use each algorithm based on market conditions

---

## Why Execution Algorithms Matter

When executing large orders, naive execution can lead to:
- **Market Impact**: Large orders move prices against you
- **Information Leakage**: Other participants detect your trading intention
- **Timing Risk**: Price moves unfavorably while you're executing

Execution algorithms aim to minimize **total execution cost** = Market Impact + Timing Risk

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from dataclasses import dataclass
from typing import List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

np.random.seed(42)
print("Libraries loaded successfully!")

---

## Part 1: Market Simulation Framework

First, let's create a realistic market simulation to test our algorithms.

In [ ]:
@dataclass
class MarketConfig:
    """Configuration for market simulation."""
    initial_price: float = 100.0
    daily_volatility: float = 0.02  # 2% daily vol
    bid_ask_spread: float = 0.01  # $0.01 spread
    avg_daily_volume: int = 1_000_000
    trading_minutes: int = 390  # 6.5 hours
    permanent_impact: float = 0.1  # Permanent price impact coefficient
    temporary_impact: float = 0.05  # Temporary price impact coefficient


class MarketSimulator:
    """Simulates intraday market with realistic volume patterns and price dynamics."""
    
    def __init__(self, config: MarketConfig):
        self.config = config
        self.minute_volatility = config.daily_volatility / np.sqrt(config.trading_minutes)
        
    def generate_volume_profile(self) -> np.ndarray:
        """
        Generate realistic U-shaped intraday volume pattern.
        Higher volume at open and close, lower in the middle.
        """
        minutes = self.config.trading_minutes
        x = np.linspace(0, 1, minutes)
        
        # U-shaped pattern using quadratic function
        base_pattern = 2.5 * (x - 0.5)**2 + 0.5
        
        # Add lunch dip (around minute 180-210)
        lunch_dip = 0.3 * np.exp(-((x - 0.5)**2) / 0.01)
        pattern = base_pattern - lunch_dip
        
        # Normalize to sum to total daily volume
        pattern = pattern / pattern.sum() * self.config.avg_daily_volume
        
        # Add noise
        noise = np.random.lognormal(0, 0.3, minutes)
        volume = pattern * noise
        
        return volume.astype(int)
    
    def generate_price_path(self, 
                           execution_schedule: Optional[np.ndarray] = None) -> pd.DataFrame:
        """
        Generate price path with optional market impact from executions.
        
        Parameters:
        -----------
        execution_schedule : array of shares executed each minute (positive = buy)
        """
        minutes = self.config.trading_minutes
        volume = self.generate_volume_profile()
        
        # Generate base price path (GBM)
        returns = np.random.normal(0, self.minute_volatility, minutes)
        
        # Add market impact if executing
        if execution_schedule is not None:
            participation_rate = execution_schedule / np.maximum(volume, 1)
            
            # Permanent impact (cumulative)
            permanent_impact = self.config.permanent_impact * np.cumsum(
                execution_schedule * np.sqrt(participation_rate)
            ) / self.config.avg_daily_volume
            
            # Temporary impact (immediate)
            temporary_impact = self.config.temporary_impact * (
                execution_schedule * participation_rate
            ) / np.sqrt(volume + 1)
            
            returns += permanent_impact / self.config.initial_price
        else:
            temporary_impact = np.zeros(minutes)
        
        # Calculate prices
        mid_prices = self.config.initial_price * np.exp(np.cumsum(returns))
        
        # Execution prices include temporary impact
        execution_prices = mid_prices + temporary_impact
        
        # Create DataFrame
        df = pd.DataFrame({
            'minute': range(minutes),
            'volume': volume,
            'mid_price': mid_prices,
            'execution_price': execution_prices,
            'bid': mid_prices - self.config.bid_ask_spread / 2,
            'ask': mid_prices + self.config.bid_ask_spread / 2
        })
        
        return df


# Test market simulator
config = MarketConfig()
simulator = MarketSimulator(config)
market_data = simulator.generate_price_path()

print(f"Market Data Shape: {market_data.shape}")
print(f"\nSample data:")
market_data.head(10)

In [ ]:
# Visualize market data
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Price path
ax1 = axes[0]
ax1.plot(market_data['minute'], market_data['mid_price'], 'b-', linewidth=1.5)
ax1.fill_between(market_data['minute'], market_data['bid'], market_data['ask'], 
                 alpha=0.3, label='Bid-Ask Spread')
ax1.set_xlabel('Minute')
ax1.set_ylabel('Price ($)')
ax1.set_title('Simulated Intraday Price Path')
ax1.legend()

# Volume profile
ax2 = axes[1]
ax2.bar(market_data['minute'], market_data['volume'], width=1, alpha=0.7, color='steelblue')
ax2.set_xlabel('Minute')
ax2.set_ylabel('Volume')
ax2.set_title('Intraday Volume Profile (U-Shaped Pattern)')

# Add time labels
time_labels = ['9:30', '10:30', '11:30', '12:30', '13:30', '14:30', '15:30', '16:00']
time_ticks = np.linspace(0, 390, len(time_labels))
for ax in axes:
    ax.set_xticks(time_ticks)
    ax.set_xticklabels(time_labels)

plt.tight_layout()
plt.show()

print(f"Total Daily Volume: {market_data['volume'].sum():,}")
print(f"Price Range: ${market_data['mid_price'].min():.2f} - ${market_data['mid_price'].max():.2f}")

---

## Part 2: TWAP (Time-Weighted Average Price)

### Theory

TWAP divides the order **equally across time intervals**, regardless of volume.

$$\text{TWAP Schedule: } q_t = \frac{Q}{T}$$

Where:
- $Q$ = Total order size
- $T$ = Number of time intervals
- $q_t$ = Shares executed at time $t$

### When to Use TWAP
- Low urgency orders
- Illiquid securities where volume prediction is unreliable
- When you want predictable, steady execution
- Benchmark: Time-weighted average market price

In [ ]:
class TWAPAlgorithm:
    """
    Time-Weighted Average Price execution algorithm.
    Distributes order evenly across time intervals.
    """
    
    def __init__(self, 
                 total_shares: int,
                 start_minute: int = 0,
                 end_minute: int = 390,
                 interval_minutes: int = 1):
        """
        Parameters:
        -----------
        total_shares : Total shares to execute
        start_minute : Start time (minute of day)
        end_minute : End time (minute of day)
        interval_minutes : Execution interval in minutes
        """
        self.total_shares = total_shares
        self.start_minute = start_minute
        self.end_minute = end_minute
        self.interval_minutes = interval_minutes
        
    def generate_schedule(self) -> np.ndarray:
        """Generate TWAP execution schedule."""
        total_minutes = 390  # Full trading day
        schedule = np.zeros(total_minutes)
        
        # Calculate number of intervals
        execution_window = self.end_minute - self.start_minute
        num_intervals = execution_window // self.interval_minutes
        
        # Shares per interval
        shares_per_interval = self.total_shares / num_intervals
        
        # Distribute evenly
        for i in range(num_intervals):
            minute = self.start_minute + i * self.interval_minutes
            if minute < total_minutes:
                schedule[minute] = shares_per_interval
        
        # Handle rounding - add remaining shares to last interval
        remaining = self.total_shares - schedule.sum()
        if remaining > 0:
            last_execution = self.start_minute + (num_intervals - 1) * self.interval_minutes
            schedule[last_execution] += remaining
            
        return schedule
    
    def calculate_benchmark(self, prices: pd.Series) -> float:
        """Calculate TWAP benchmark price."""
        window_prices = prices.iloc[self.start_minute:self.end_minute]
        return window_prices.mean()


# Example: Execute 100,000 shares using TWAP over full day
twap = TWAPAlgorithm(
    total_shares=100_000,
    start_minute=0,
    end_minute=390,
    interval_minutes=5  # Execute every 5 minutes
)

twap_schedule = twap.generate_schedule()

print(f"TWAP Execution Schedule Summary:")
print(f"Total shares: {twap_schedule.sum():,.0f}")
print(f"Number of executions: {(twap_schedule > 0).sum()}")
print(f"Shares per execution: {twap_schedule[twap_schedule > 0].mean():,.0f}")

In [ ]:
# Visualize TWAP schedule
fig, ax = plt.subplots(figsize=(14, 5))

execution_minutes = np.where(twap_schedule > 0)[0]
execution_shares = twap_schedule[execution_minutes]

ax.bar(execution_minutes, execution_shares, width=3, alpha=0.7, color='green', label='TWAP Executions')
ax.axhline(y=execution_shares.mean(), color='red', linestyle='--', label=f'Avg: {execution_shares.mean():,.0f} shares')

ax.set_xlabel('Minute')
ax.set_ylabel('Shares')
ax.set_title('TWAP Execution Schedule - Equal Distribution Over Time')
ax.legend()

# Time labels
time_labels = ['9:30', '10:30', '11:30', '12:30', '13:30', '14:30', '15:30', '16:00']
time_ticks = np.linspace(0, 390, len(time_labels))
ax.set_xticks(time_ticks)
ax.set_xticklabels(time_labels)

plt.tight_layout()
plt.show()

---

## Part 3: VWAP (Volume-Weighted Average Price)

### Theory

VWAP distributes the order **proportional to expected volume** at each interval.

$$\text{VWAP Schedule: } q_t = Q \cdot \frac{v_t}{\sum_{i=1}^T v_i}$$

Where:
- $Q$ = Total order size  
- $v_t$ = Expected volume at time $t$
- $q_t$ = Shares executed at time $t$

### VWAP Benchmark

$$\text{VWAP} = \frac{\sum_{t=1}^T P_t \cdot V_t}{\sum_{t=1}^T V_t}$$

### When to Use VWAP
- Medium urgency orders
- When matching market VWAP is the benchmark
- Want to minimize market impact by trading with volume
- Liquid securities with predictable volume patterns

In [ ]:
class VWAPAlgorithm:
    """
    Volume-Weighted Average Price execution algorithm.
    Distributes order proportional to expected volume.
    """
    
    def __init__(self,
                 total_shares: int,
                 start_minute: int = 0,
                 end_minute: int = 390,
                 max_participation_rate: float = 0.1):
        """
        Parameters:
        -----------
        total_shares : Total shares to execute
        start_minute : Start time (minute of day)
        end_minute : End time (minute of day)  
        max_participation_rate : Maximum % of volume to capture per interval
        """
        self.total_shares = total_shares
        self.start_minute = start_minute
        self.end_minute = end_minute
        self.max_participation_rate = max_participation_rate
        
    def generate_schedule(self, volume_forecast: np.ndarray) -> np.ndarray:
        """
        Generate VWAP execution schedule based on volume forecast.
        
        Parameters:
        -----------
        volume_forecast : Expected volume for each minute
        """
        schedule = np.zeros(len(volume_forecast))
        
        # Get volume in execution window
        window_volume = volume_forecast[self.start_minute:self.end_minute]
        total_window_volume = window_volume.sum()
        
        # Calculate participation weights
        weights = window_volume / total_window_volume
        
        # Distribute shares according to volume weights
        schedule[self.start_minute:self.end_minute] = self.total_shares * weights
        
        # Apply participation rate cap
        max_shares = volume_forecast * self.max_participation_rate
        schedule = np.minimum(schedule, max_shares)
        
        # Scale to match total (after capping)
        if schedule.sum() > 0:
            scale = self.total_shares / schedule.sum()
            schedule *= scale
            
        return schedule
    
    @staticmethod
    def calculate_benchmark(prices: pd.Series, volumes: pd.Series) -> float:
        """Calculate VWAP benchmark price."""
        return (prices * volumes).sum() / volumes.sum()
    
    @staticmethod
    def calculate_realized_vwap(execution_prices: np.ndarray, 
                                execution_shares: np.ndarray) -> float:
        """Calculate realized execution VWAP."""
        mask = execution_shares > 0
        if mask.sum() == 0:
            return 0
        return (execution_prices[mask] * execution_shares[mask]).sum() / execution_shares[mask].sum()


# Example: Execute 100,000 shares using VWAP
vwap = VWAPAlgorithm(
    total_shares=100_000,
    start_minute=0,
    end_minute=390,
    max_participation_rate=0.15
)

# Use historical volume pattern as forecast
volume_forecast = market_data['volume'].values
vwap_schedule = vwap.generate_schedule(volume_forecast)

print(f"VWAP Execution Schedule Summary:")
print(f"Total shares: {vwap_schedule.sum():,.0f}")
print(f"Number of execution minutes: {(vwap_schedule > 0).sum()}")
print(f"Max shares in single minute: {vwap_schedule.max():,.0f}")

In [ ]:
# Compare TWAP vs VWAP schedules
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Top: Schedules comparison
ax1 = axes[0]
ax1.fill_between(range(390), twap_schedule, alpha=0.5, label='TWAP', color='blue')
ax1.fill_between(range(390), vwap_schedule, alpha=0.5, label='VWAP', color='green')
ax1.set_xlabel('Minute')
ax1.set_ylabel('Shares per Minute')
ax1.set_title('TWAP vs VWAP Execution Schedules')
ax1.legend()

# Bottom: Volume overlay
ax2 = axes[1]
ax2.bar(range(390), volume_forecast, width=1, alpha=0.3, color='gray', label='Market Volume')
ax2.plot(range(390), vwap_schedule * 20, 'g-', linewidth=2, label='VWAP Schedule (scaled)')
ax2.set_xlabel('Minute')
ax2.set_ylabel('Volume')
ax2.set_title('VWAP Schedule Follows Volume Pattern')
ax2.legend()

# Time labels
time_labels = ['9:30', '10:30', '11:30', '12:30', '13:30', '14:30', '15:30', '16:00']
time_ticks = np.linspace(0, 390, len(time_labels))
for ax in axes:
    ax.set_xticks(time_ticks)
    ax.set_xticklabels(time_labels)

plt.tight_layout()
plt.show()

---

## Part 4: Implementation Shortfall (IS)

### Theory

Implementation Shortfall (IS), also known as **Arrival Price** algorithm, minimizes the total cost of execution relative to the price when the decision to trade was made.

$$\text{Implementation Shortfall} = \text{Paper Return} - \text{Actual Return}$$

### Components of IS:

1. **Execution Cost** = (Execution Price - Decision Price) × Shares
2. **Delay Cost** = Price drift before execution starts
3. **Market Impact** = Price move caused by our trading
4. **Timing Cost** = Price drift during execution
5. **Opportunity Cost** = Unfilled portion × Price move

### Almgren-Chriss Model

The optimal IS strategy balances:
- **Market Impact** (trade fast → high impact)
- **Volatility Risk** (trade slow → exposed to price moves)

Optimal trajectory:
$$x_t = X \cdot \frac{\sinh(\kappa(T-t))}{\sinh(\kappa T)}$$

Where $\kappa = \sqrt{\frac{\lambda \sigma^2}{\eta}}$ is the urgency parameter.

In [ ]:
class ImplementationShortfallAlgorithm:
    """
    Implementation Shortfall execution algorithm.
    Based on Almgren-Chriss optimal execution model.
    """
    
    def __init__(self,
                 total_shares: int,
                 start_minute: int = 0,
                 end_minute: int = 390,
                 volatility: float = 0.02,
                 risk_aversion: float = 1e-6,
                 temporary_impact: float = 0.05,
                 permanent_impact: float = 0.1):
        """
        Parameters:
        -----------
        total_shares : Total shares to execute
        start_minute : Start time (minute of day)
        end_minute : End time (minute of day)
        volatility : Daily volatility (sigma)
        risk_aversion : Risk aversion parameter (lambda)
        temporary_impact : Temporary impact coefficient (eta)
        permanent_impact : Permanent impact coefficient (gamma)
        """
        self.total_shares = total_shares
        self.start_minute = start_minute
        self.end_minute = end_minute
        self.volatility = volatility
        self.risk_aversion = risk_aversion
        self.temporary_impact = temporary_impact
        self.permanent_impact = permanent_impact
        
        # Calculate minute volatility
        self.minute_vol = volatility / np.sqrt(390)
        
    def calculate_urgency(self) -> float:
        """
        Calculate Almgren-Chriss urgency parameter kappa.
        Higher kappa = trade faster (more aggressive)
        """
        kappa = np.sqrt(
            self.risk_aversion * self.minute_vol**2 / self.temporary_impact
        )
        return kappa
    
    def generate_schedule(self, urgency_override: Optional[float] = None) -> np.ndarray:
        """
        Generate optimal IS execution schedule using Almgren-Chriss.
        
        Parameters:
        -----------
        urgency_override : Override calculated urgency (for sensitivity analysis)
        """
        schedule = np.zeros(390)
        
        T = self.end_minute - self.start_minute
        kappa = urgency_override if urgency_override else self.calculate_urgency()
        
        # Calculate remaining holdings at each time
        # x(t) = X * sinh(kappa*(T-t)) / sinh(kappa*T)
        remaining = np.zeros(T + 1)
        remaining[0] = self.total_shares
        
        for t in range(T):
            # Remaining shares at time t
            remaining[t] = self.total_shares * (
                np.sinh(kappa * (T - t)) / np.sinh(kappa * T)
            )
        remaining[T] = 0
        
        # Execution rate = -dx/dt (shares sold at each interval)
        for t in range(T):
            schedule[self.start_minute + t] = remaining[t] - remaining[t + 1]
            
        return schedule
    
    def calculate_shortfall(self,
                           decision_price: float,
                           execution_prices: np.ndarray,
                           execution_shares: np.ndarray,
                           final_price: float) -> dict:
        """
        Calculate implementation shortfall breakdown.
        
        Parameters:
        -----------
        decision_price : Price at decision time (arrival price)
        execution_prices : Prices at each execution
        execution_shares : Shares executed at each time
        final_price : Final market price
        """
        filled_shares = execution_shares.sum()
        unfilled_shares = self.total_shares - filled_shares
        
        # Average execution price
        mask = execution_shares > 0
        if mask.sum() == 0:
            avg_exec_price = decision_price
        else:
            avg_exec_price = (
                (execution_prices[mask] * execution_shares[mask]).sum() / 
                execution_shares[mask].sum()
            )
        
        # Paper portfolio value (if executed at decision price)
        paper_value = self.total_shares * (final_price - decision_price)
        
        # Actual portfolio value
        execution_cost = filled_shares * (avg_exec_price - decision_price)
        remaining_value = unfilled_shares * (final_price - decision_price)
        actual_value = remaining_value - execution_cost
        
        # Implementation shortfall
        total_is = paper_value - actual_value
        is_bps = (total_is / (self.total_shares * decision_price)) * 10000
        
        return {
            'decision_price': decision_price,
            'avg_execution_price': avg_exec_price,
            'final_price': final_price,
            'filled_shares': filled_shares,
            'unfilled_shares': unfilled_shares,
            'paper_value': paper_value,
            'actual_value': actual_value,
            'total_shortfall': total_is,
            'shortfall_bps': is_bps
        }


# Example: IS algorithm with different urgency levels
is_algo = ImplementationShortfallAlgorithm(
    total_shares=100_000,
    start_minute=0,
    end_minute=390,
    volatility=0.02,
    risk_aversion=1e-6
)

print(f"Calculated urgency (kappa): {is_algo.calculate_urgency():.6f}")

# Generate schedules with different urgency levels
is_schedule_low = is_algo.generate_schedule(urgency_override=0.005)
is_schedule_mid = is_algo.generate_schedule(urgency_override=0.02)
is_schedule_high = is_algo.generate_schedule(urgency_override=0.1)

print(f"\nSchedule Statistics:")
print(f"Low urgency  - Peak: {is_schedule_low.max():,.0f}, Duration: {(is_schedule_low > 10).sum()} mins")
print(f"Mid urgency  - Peak: {is_schedule_mid.max():,.0f}, Duration: {(is_schedule_mid > 10).sum()} mins")
print(f"High urgency - Peak: {is_schedule_high.max():,.0f}, Duration: {(is_schedule_high > 10).sum()} mins")

In [ ]:
# Visualize IS schedules with different urgency
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Execution rate
ax1 = axes[0]
ax1.plot(is_schedule_low, label='Low Urgency (κ=0.005)', linewidth=2)
ax1.plot(is_schedule_mid, label='Medium Urgency (κ=0.02)', linewidth=2)
ax1.plot(is_schedule_high, label='High Urgency (κ=0.1)', linewidth=2)
ax1.set_xlabel('Minute')
ax1.set_ylabel('Shares per Minute')
ax1.set_title('IS Algorithm: Execution Rate by Urgency')
ax1.legend()

# Right: Cumulative shares
ax2 = axes[1]
ax2.plot(np.cumsum(is_schedule_low), label='Low Urgency', linewidth=2)
ax2.plot(np.cumsum(is_schedule_mid), label='Medium Urgency', linewidth=2)
ax2.plot(np.cumsum(is_schedule_high), label='High Urgency', linewidth=2)
ax2.axhline(y=100_000, color='black', linestyle='--', alpha=0.5, label='Target')
ax2.set_xlabel('Minute')
ax2.set_ylabel('Cumulative Shares Executed')
ax2.set_title('IS Algorithm: Execution Progress by Urgency')
ax2.legend()

# Time labels
time_labels = ['9:30', '10:30', '11:30', '12:30', '13:30', '14:30', '15:30', '16:00']
time_ticks = np.linspace(0, 390, len(time_labels))
for ax in axes:
    ax.set_xticks(time_ticks)
    ax.set_xticklabels(time_labels)

plt.tight_layout()
plt.show()

---

## Part 5: Execution Simulation & Performance Comparison

Let's simulate executing orders using each algorithm and compare performance.

In [ ]:
class ExecutionSimulator:
    """
    Simulates order execution and calculates performance metrics.
    """
    
    def __init__(self, market_simulator: MarketSimulator):
        self.market_sim = market_simulator
        
    def execute_order(self, schedule: np.ndarray) -> dict:
        """
        Execute order according to schedule and return results.
        """
        # Generate market with impact from our execution
        market_data = self.market_sim.generate_price_path(schedule)
        
        # Calculate execution metrics
        mask = schedule > 0
        execution_prices = market_data['execution_price'].values
        mid_prices = market_data['mid_price'].values
        volumes = market_data['volume'].values
        
        # Average execution price
        avg_exec_price = (
            (execution_prices[mask] * schedule[mask]).sum() / schedule[mask].sum()
        )
        
        # Benchmarks
        arrival_price = mid_prices[0]
        twap_benchmark = mid_prices.mean()
        vwap_benchmark = (mid_prices * volumes).sum() / volumes.sum()
        close_price = mid_prices[-1]
        
        # Performance vs benchmarks (in bps)
        vs_arrival = (avg_exec_price - arrival_price) / arrival_price * 10000
        vs_twap = (avg_exec_price - twap_benchmark) / twap_benchmark * 10000
        vs_vwap = (avg_exec_price - vwap_benchmark) / vwap_benchmark * 10000
        vs_close = (avg_exec_price - close_price) / close_price * 10000
        
        # Participation rate
        participation = schedule / np.maximum(volumes, 1)
        avg_participation = participation[mask].mean() * 100
        max_participation = participation.max() * 100
        
        return {
            'market_data': market_data,
            'schedule': schedule,
            'avg_exec_price': avg_exec_price,
            'arrival_price': arrival_price,
            'twap_benchmark': twap_benchmark,
            'vwap_benchmark': vwap_benchmark,
            'close_price': close_price,
            'vs_arrival_bps': vs_arrival,
            'vs_twap_bps': vs_twap,
            'vs_vwap_bps': vs_vwap,
            'vs_close_bps': vs_close,
            'avg_participation_pct': avg_participation,
            'max_participation_pct': max_participation,
            'total_shares': schedule.sum(),
            'num_executions': mask.sum()
        }


# Run simulation
config = MarketConfig(initial_price=100, daily_volatility=0.02)
market_sim = MarketSimulator(config)
exec_sim = ExecutionSimulator(market_sim)

# Generate schedules for 100k shares
total_shares = 100_000

# TWAP
twap_algo = TWAPAlgorithm(total_shares, interval_minutes=5)
twap_schedule = twap_algo.generate_schedule()

# VWAP (need volume forecast)
dummy_market = market_sim.generate_price_path()
volume_forecast = dummy_market['volume'].values
vwap_algo = VWAPAlgorithm(total_shares, max_participation_rate=0.15)
vwap_schedule = vwap_algo.generate_schedule(volume_forecast)

# IS with medium urgency
is_algo = ImplementationShortfallAlgorithm(total_shares, risk_aversion=5e-7)
is_schedule = is_algo.generate_schedule()

print("Schedules generated. Running simulations...")

In [ ]:
# Run Monte Carlo simulation to compare algorithms
n_simulations = 100

results = {
    'TWAP': [],
    'VWAP': [],
    'IS': []
}

for i in range(n_simulations):
    # TWAP
    twap_result = exec_sim.execute_order(twap_schedule)
    results['TWAP'].append(twap_result)
    
    # VWAP (regenerate volume forecast for each sim)
    new_volume = market_sim.generate_volume_profile()
    vwap_schedule = vwap_algo.generate_schedule(new_volume)
    vwap_result = exec_sim.execute_order(vwap_schedule)
    results['VWAP'].append(vwap_result)
    
    # IS
    is_result = exec_sim.execute_order(is_schedule)
    results['IS'].append(is_result)

print(f"Completed {n_simulations} simulations for each algorithm.")

In [ ]:
# Analyze results
def summarize_results(results_list: list) -> pd.Series:
    """Summarize execution results."""
    metrics = {
        'Avg Exec Price': np.mean([r['avg_exec_price'] for r in results_list]),
        'vs Arrival (bps)': np.mean([r['vs_arrival_bps'] for r in results_list]),
        'vs TWAP (bps)': np.mean([r['vs_twap_bps'] for r in results_list]),
        'vs VWAP (bps)': np.mean([r['vs_vwap_bps'] for r in results_list]),
        'Std vs Arrival (bps)': np.std([r['vs_arrival_bps'] for r in results_list]),
        'Avg Participation (%)': np.mean([r['avg_participation_pct'] for r in results_list]),
        'Max Participation (%)': np.mean([r['max_participation_pct'] for r in results_list]),
    }
    return pd.Series(metrics)


# Create summary table
summary = pd.DataFrame({
    algo: summarize_results(res) for algo, res in results.items()
}).T

print("\n" + "="*70)
print("EXECUTION ALGORITHM COMPARISON - Monte Carlo Results")
print("="*70)
print(f"\nOrder Size: {total_shares:,} shares")
print(f"Simulations: {n_simulations}")
print(f"\n{summary.round(2).to_string()}")

In [ ]:
# Visualize performance distributions
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Extract metrics for plotting
metrics_data = {}
for algo in ['TWAP', 'VWAP', 'IS']:
    metrics_data[algo] = {
        'vs_arrival': [r['vs_arrival_bps'] for r in results[algo]],
        'vs_vwap': [r['vs_vwap_bps'] for r in results[algo]],
        'participation': [r['avg_participation_pct'] for r in results[algo]]
    }

# Plot 1: Distribution vs Arrival Price
ax1 = axes[0, 0]
for algo, color in zip(['TWAP', 'VWAP', 'IS'], ['blue', 'green', 'red']):
    ax1.hist(metrics_data[algo]['vs_arrival'], bins=20, alpha=0.5, 
             label=algo, color=color)
ax1.axvline(x=0, color='black', linestyle='--', alpha=0.7)
ax1.set_xlabel('Slippage vs Arrival (bps)')
ax1.set_ylabel('Frequency')
ax1.set_title('Distribution: Performance vs Arrival Price')
ax1.legend()

# Plot 2: Distribution vs VWAP
ax2 = axes[0, 1]
for algo, color in zip(['TWAP', 'VWAP', 'IS'], ['blue', 'green', 'red']):
    ax2.hist(metrics_data[algo]['vs_vwap'], bins=20, alpha=0.5,
             label=algo, color=color)
ax2.axvline(x=0, color='black', linestyle='--', alpha=0.7)
ax2.set_xlabel('Slippage vs VWAP (bps)')
ax2.set_ylabel('Frequency')
ax2.set_title('Distribution: Performance vs VWAP Benchmark')
ax2.legend()

# Plot 3: Box plot comparison
ax3 = axes[1, 0]
box_data = [metrics_data[algo]['vs_arrival'] for algo in ['TWAP', 'VWAP', 'IS']]
bp = ax3.boxplot(box_data, labels=['TWAP', 'VWAP', 'IS'], patch_artist=True)
colors = ['lightblue', 'lightgreen', 'lightcoral']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
ax3.axhline(y=0, color='black', linestyle='--', alpha=0.7)
ax3.set_ylabel('Slippage vs Arrival (bps)')
ax3.set_title('Box Plot: Algorithm Performance Comparison')

# Plot 4: Risk-Return (Mean vs Std)
ax4 = axes[1, 1]
for algo, marker in zip(['TWAP', 'VWAP', 'IS'], ['o', 's', '^']):
    mean_slip = np.mean(metrics_data[algo]['vs_arrival'])
    std_slip = np.std(metrics_data[algo]['vs_arrival'])
    ax4.scatter(std_slip, mean_slip, s=200, marker=marker, label=algo)
    ax4.annotate(algo, (std_slip, mean_slip), fontsize=12, 
                 xytext=(5, 5), textcoords='offset points')
ax4.axhline(y=0, color='black', linestyle='--', alpha=0.3)
ax4.set_xlabel('Volatility of Slippage (bps)')
ax4.set_ylabel('Mean Slippage (bps)')
ax4.set_title('Risk-Return: Mean vs Volatility of Execution Cost')
ax4.legend()

plt.tight_layout()
plt.show()

---

## Part 6: Advanced Concepts

### 6.1 Adaptive VWAP

Real VWAP algorithms adapt to actual vs expected volume in real-time.

In [ ]:
class AdaptiveVWAP:
    """
    Adaptive VWAP that adjusts based on realized vs expected volume.
    """
    
    def __init__(self,
                 total_shares: int,
                 volume_forecast: np.ndarray,
                 max_participation: float = 0.15,
                 catchup_aggression: float = 1.5):
        self.total_shares = total_shares
        self.volume_forecast = volume_forecast
        self.max_participation = max_participation
        self.catchup_aggression = catchup_aggression
        
        # State tracking
        self.shares_executed = 0
        self.current_minute = 0
        
    def get_target_completion(self, minute: int) -> float:
        """Calculate target completion % based on volume forecast."""
        cum_forecast = np.cumsum(self.volume_forecast)
        total_forecast = cum_forecast[-1]
        return cum_forecast[minute] / total_forecast
    
    def calculate_order(self, minute: int, actual_volume: float) -> float:
        """
        Calculate shares to execute this minute.
        
        Parameters:
        -----------
        minute : Current minute
        actual_volume : Actual market volume this minute
        """
        remaining_shares = self.total_shares - self.shares_executed
        if remaining_shares <= 0:
            return 0
            
        # Calculate target vs actual completion
        target_completion = self.get_target_completion(minute)
        actual_completion = self.shares_executed / self.total_shares
        completion_gap = target_completion - actual_completion
        
        # Base order from volume forecast
        base_participation = self.volume_forecast[minute] / np.sum(self.volume_forecast[minute:])
        base_order = remaining_shares * base_participation
        
        # Adjust for completion gap
        if completion_gap > 0:  # Behind schedule
            adjustment = 1 + self.catchup_aggression * completion_gap
        else:  # Ahead of schedule
            adjustment = 1 + completion_gap  # Slow down
            
        adjusted_order = base_order * adjustment
        
        # Apply participation cap
        max_order = actual_volume * self.max_participation
        final_order = min(adjusted_order, max_order, remaining_shares)
        
        # Update state
        self.shares_executed += final_order
        self.current_minute = minute
        
        return final_order
    
    def reset(self):
        """Reset algorithm state."""
        self.shares_executed = 0
        self.current_minute = 0


# Demonstrate adaptive VWAP
volume_forecast = market_sim.generate_volume_profile()
actual_volume = market_sim.generate_volume_profile()  # Different realization

adaptive_vwap = AdaptiveVWAP(
    total_shares=100_000,
    volume_forecast=volume_forecast,
    max_participation=0.15,
    catchup_aggression=2.0
)

# Simulate execution
adaptive_schedule = []
target_completion = []
actual_completion = []

for minute in range(390):
    order = adaptive_vwap.calculate_order(minute, actual_volume[minute])
    adaptive_schedule.append(order)
    target_completion.append(adaptive_vwap.get_target_completion(minute))
    actual_completion.append(adaptive_vwap.shares_executed / 100_000)

adaptive_schedule = np.array(adaptive_schedule)

print(f"Adaptive VWAP Results:")
print(f"Total executed: {adaptive_schedule.sum():,.0f}")
print(f"Final completion: {actual_completion[-1]:.1%}")

In [ ]:
# Visualize adaptive VWAP tracking
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Completion tracking
ax1 = axes[0]
ax1.plot(target_completion, 'b--', label='Target Completion (from forecast)', linewidth=2)
ax1.plot(actual_completion, 'g-', label='Actual Completion', linewidth=2)
ax1.fill_between(range(390), target_completion, actual_completion, alpha=0.3)
ax1.set_xlabel('Minute')
ax1.set_ylabel('Completion %')
ax1.set_title('Adaptive VWAP: Target vs Actual Completion')
ax1.legend()

# Volume comparison
ax2 = axes[1]
ax2.bar(range(390), actual_volume, width=1, alpha=0.3, label='Actual Market Volume', color='gray')
ax2.bar(range(390), adaptive_schedule, width=1, alpha=0.7, label='Adaptive Orders', color='green')
ax2.set_xlabel('Minute')
ax2.set_ylabel('Shares')
ax2.set_title('Adaptive VWAP: Orders Track Actual Volume')
ax2.legend()

plt.tight_layout()
plt.show()

### 6.2 Participation of Volume (POV) Algorithm

POV executes at a fixed percentage of market volume.

In [ ]:
class POVAlgorithm:
    """
    Participation of Volume algorithm.
    Executes at a fixed % of market volume.
    """
    
    def __init__(self,
                 total_shares: int,
                 target_participation: float = 0.10,
                 min_participation: float = 0.05,
                 max_participation: float = 0.20):
        self.total_shares = total_shares
        self.target_participation = target_participation
        self.min_participation = min_participation
        self.max_participation = max_participation
        
    def generate_schedule(self, actual_volumes: np.ndarray) -> np.ndarray:
        """
        Generate POV schedule based on actual volumes.
        """
        schedule = np.zeros(len(actual_volumes))
        remaining = self.total_shares
        
        for t, vol in enumerate(actual_volumes):
            if remaining <= 0:
                break
                
            # Calculate order as % of volume
            order = vol * self.target_participation
            
            # Apply constraints
            order = min(order, remaining)
            order = min(order, vol * self.max_participation)
            order = max(order, vol * self.min_participation) if remaining > 0 else 0
            order = min(order, remaining)
            
            schedule[t] = order
            remaining -= order
            
        return schedule


# Compare POV with different participation rates
pov_schedules = {}
for participation in [0.05, 0.10, 0.20]:
    pov = POVAlgorithm(100_000, target_participation=participation)
    pov_schedules[f'{participation:.0%}'] = pov.generate_schedule(volume_forecast)

# Visualize
fig, ax = plt.subplots(figsize=(14, 5))

for label, schedule in pov_schedules.items():
    cum_schedule = np.cumsum(schedule)
    ax.plot(cum_schedule, label=f'POV {label}', linewidth=2)

ax.axhline(y=100_000, color='black', linestyle='--', alpha=0.5, label='Target')
ax.set_xlabel('Minute')
ax.set_ylabel('Cumulative Shares')
ax.set_title('POV Algorithm: Completion Time Depends on Participation Rate')
ax.legend()

time_labels = ['9:30', '10:30', '11:30', '12:30', '13:30', '14:30', '15:30', '16:00']
ax.set_xticks(np.linspace(0, 390, len(time_labels)))
ax.set_xticklabels(time_labels)

plt.tight_layout()
plt.show()

for label, schedule in pov_schedules.items():
    completion_time = np.argmax(np.cumsum(schedule) >= 100_000)
    print(f"POV {label}: Completes at minute {completion_time} ({completion_time // 60}h {completion_time % 60}m)")

---

## Part 7: Algorithm Selection Guide

### Decision Framework

In [ ]:
# Algorithm Selection Matrix
selection_guide = pd.DataFrame({
    'TWAP': {
        'Best For': 'Low urgency, illiquid stocks',
        'Benchmark': 'Time-weighted average price',
        'Market Impact': 'Medium (constant)',
        'Timing Risk': 'Low (distributed)',
        'Complexity': 'Simple',
        'Use When': 'Volume patterns unpredictable'
    },
    'VWAP': {
        'Best For': 'Medium urgency, liquid stocks',
        'Benchmark': 'Volume-weighted average price',
        'Market Impact': 'Low (follows volume)',
        'Timing Risk': 'Medium',
        'Complexity': 'Medium',
        'Use When': 'Want to match market VWAP'
    },
    'IS': {
        'Best For': 'High urgency, volatile markets',
        'Benchmark': 'Arrival/Decision price',
        'Market Impact': 'Varies (front-loaded)',
        'Timing Risk': 'Minimized',
        'Complexity': 'High',
        'Use When': 'Price momentum or high vol'
    },
    'POV': {
        'Best For': 'Stealth execution',
        'Benchmark': 'Participation rate',
        'Market Impact': 'Low (constant %)',
        'Timing Risk': 'Variable',
        'Complexity': 'Medium',
        'Use When': 'Want to avoid detection'
    }
}).T

print("\n" + "="*80)
print("EXECUTION ALGORITHM SELECTION GUIDE")
print("="*80)
print(selection_guide.to_string())

In [ ]:
# Visualize algorithm characteristics
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Urgency vs Impact tradeoff
ax1 = axes[0]
algos = ['TWAP', 'VWAP', 'IS (Low)', 'IS (High)', 'POV']
urgency = [3, 5, 2, 9, 4]
impact = [5, 3, 2, 7, 3]
colors = ['blue', 'green', 'red', 'darkred', 'purple']

for i, (algo, u, imp, c) in enumerate(zip(algos, urgency, impact, colors)):
    ax1.scatter(u, imp, s=300, c=c, label=algo, alpha=0.7)
    ax1.annotate(algo, (u, imp), fontsize=10, ha='center', va='bottom',
                 xytext=(0, 10), textcoords='offset points')

ax1.set_xlabel('Urgency / Speed', fontsize=12)
ax1.set_ylabel('Market Impact', fontsize=12)
ax1.set_title('Algorithm Tradeoff: Speed vs Impact')
ax1.set_xlim(0, 10)
ax1.set_ylim(0, 10)

# Execution profiles
ax2 = axes[1]
minutes = np.arange(390)

# Normalized schedules
ax2.plot(twap_schedule / twap_schedule.max(), label='TWAP', alpha=0.7)
ax2.plot(vwap_schedule / vwap_schedule.max(), label='VWAP', alpha=0.7)
ax2.plot(is_schedule / is_schedule.max(), label='IS', alpha=0.7)

ax2.set_xlabel('Minute')
ax2.set_ylabel('Normalized Execution Rate')
ax2.set_title('Execution Profiles Comparison')
ax2.legend()

plt.tight_layout()
plt.show()

---

## Part 8: Practice Exercises

### Exercise 1: Implement a Hybrid Algorithm
Create an algorithm that:
- Uses IS for the first 30% of the order (capture favorable price)
- Switches to VWAP for the remaining 70% (minimize impact)

In [ ]:
# Exercise 1: Your implementation here
class HybridISVWAP:
    """
    Hybrid algorithm: IS for urgency, VWAP for completion.
    """
    
    def __init__(self,
                 total_shares: int,
                 is_fraction: float = 0.3,
                 is_urgency: float = 0.05):
        self.total_shares = total_shares
        self.is_fraction = is_fraction
        self.is_shares = int(total_shares * is_fraction)
        self.vwap_shares = total_shares - self.is_shares
        self.is_urgency = is_urgency
        
    def generate_schedule(self, volume_forecast: np.ndarray) -> np.ndarray:
        """Generate hybrid schedule."""
        # TODO: Implement the hybrid algorithm
        # Hint: Use IS for first portion, VWAP for rest
        schedule = np.zeros(390)
        
        # Your code here
        
        return schedule

# Test your implementation
# hybrid = HybridISVWAP(100_000, is_fraction=0.3)
# hybrid_schedule = hybrid.generate_schedule(volume_forecast)

### Exercise 2: Market Impact Analysis
Analyze how execution cost varies with:
- Order size (50k, 100k, 200k shares)
- Participation rate
- Volatility

In [ ]:
# Exercise 2: Your analysis here
# TODO: Run simulations with different parameters and plot the results

# Hint: Create a parameter grid
order_sizes = [50_000, 100_000, 200_000]
volatilities = [0.01, 0.02, 0.04]

# Your analysis code here

### Exercise 3: Transaction Cost Analysis (TCA)
Build a TCA report that includes:
- Arrival price slippage
- VWAP slippage  
- Market impact estimate
- Timing cost

In [ ]:
# Exercise 3: Your TCA implementation here
def generate_tca_report(execution_result: dict) -> pd.DataFrame:
    """
    Generate a Transaction Cost Analysis report.
    
    Parameters:
    -----------
    execution_result : Output from ExecutionSimulator.execute_order()
    
    Returns:
    --------
    DataFrame with TCA metrics
    """
    # TODO: Implement TCA report
    # Include: arrival slippage, VWAP slippage, estimated market impact, timing cost
    
    report = {}
    
    # Your code here
    
    return pd.DataFrame([report])

# Test with a sample execution
# tca_report = generate_tca_report(twap_result)
# print(tca_report)

---

## Summary

### Key Takeaways

1. **TWAP** - Simple, predictable, best for illiquid/unpredictable markets
2. **VWAP** - Follows volume, minimizes detection, matches market benchmark
3. **Implementation Shortfall** - Balances urgency vs impact, front-loads execution
4. **POV** - Maintains constant market participation, good for stealth

### Cost Components
- **Market Impact**: Direct cost from our trading pressure
- **Timing Risk**: Exposure to adverse price moves
- **Opportunity Cost**: Missed execution (unfilled shares)

### Real-World Considerations
- Algorithm selection depends on market conditions
- Adaptive algorithms outperform static ones
- TCA is essential for measuring execution quality
- Dark pools and alternative venues add complexity

---

## References

1. Almgren, R. & Chriss, N. (2000). "Optimal Execution of Portfolio Transactions"
2. Kissell, R. (2013). "The Science of Algorithmic Trading and Portfolio Management"
3. Cartea, Á., Jaimungal, S., & Penalva, J. (2015). "Algorithmic and High-Frequency Trading"